STEP-0

In [ ]:
!pip install pandas scikit-learn numpy nltk gensim transformers torch
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.0 MB/s eta 0:00:00


STEP-1

In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/FinalHindiDataset_withEmojis.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Display a summary of the data
print(data.info())
print(data.head())

data.sample(n=100);


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9907 entries, 0 to 9906
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       9907 non-null   int64  
 1   Label            9899 non-null   float64
 2   Aug_text         9907 non-null   object 
 3   translated_text  9907 non-null   object 
 4   word_count       9907 non-null   int64  
 5   Emojis           9907 non-null   object 
 6   Unnamed: 6       0 non-null      float64
 7   Unnamed: 7       0 non-null      float64
 8   Unnamed: 8       0 non-null      float64
dtypes: float64(4), int64(2), object(3)
memory usage: 696.7+ KB
None
   Unnamed: 0  Label                                           Aug_text  \
0           0    0.0  3. A day of historic importance and national j...   
1           1    0.0  4. A momentous occasion for India - Prime Mini...   
2           2    0.0   nobody else has taken over as the Home Minist...   
3           3    0.

STEP-2: performing LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Preprocess the English text
english_text = data['Aug_text'].fillna("").tolist()

# Convert text to a document-term matrix
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(english_text)

# Fit the LDA model
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda_features = lda.fit_transform(dtm)

# Add LDA features to the dataframe
lda_df = pd.DataFrame(lda_features, columns=[f"LDA_Topic_{i}" for i in range(10)])
data = pd.concat([data, lda_df], axis=1)


NER: named entity relationship

In [ ]:
!pip install stanza

In [ ]:
# # !pip install stanza
# import stanza
# stanza.download('hi')  # Download the Hindi model

# # Initialize Stanza pipeline for Hindi
# nlp = stanza.Pipeline('hi')

# # Function to extract named entities
# def extract_named_entities_stanza(text):
#     doc = nlp(text)
#     entities = [ent.text for sent in doc.sentences for ent in sent.ents]
#     return len(entities)  # Count the named entities

# # Apply NER on Hindi text
# data['NER_Count'] = data['translated_text'].fillna("").apply(extract_named_entities_stanza)
# print(data.info())


STEP-3:Fact-checking

In [ ]:
# import pandas as pd

# # Load the LIAR dataset (train.tsv)
# liar_df = pd.read_csv('/content/liar_train.tsv', sep='\t', header=None)

# # Check the first few rows to understand its structure
# print(liar_df.head())

# # The columns might not have headers, so we will manually assign them
# liar_df.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'speaker_job', 'stateinfo', 'party', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'true_counts', 'class']

# # Now extract only the 'statement' column for fact-checking
# fact_check_corpus = liar_df['statement'].dropna().tolist()

# # Check the first few statements
# print(fact_check_corpus[:5])


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch

# # Load the pre-trained tokenizer and model
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Function to ensure proper tokenization and embedding extraction
# def get_embeddings(text):
#     # Tokenize with truncation and padding
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

#     # Forward pass through the model to get token embeddings
#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Extract the embeddings for the [CLS] token (first token) or average pooling of token embeddings
#     # CLS token embedding: outputs[0] is of shape (batch_size, sequence_length, hidden_size)
#     embeddings = outputs.last_hidden_state[0][0]  # [CLS] token embedding (size: hidden_size)

#     # Alternatively, if you want to average over all token embeddings:
#     # embeddings = outputs.last_hidden_state[0].mean(dim=0)

#     return embeddings.numpy()  # Convert to numpy array

# # Example usage for fact-check corpus
# fact_check_corpus = liar_df['statement'].dropna().tolist()

# # Convert LIAR fact-check claims into embeddings
# fact_embeddings = [get_embeddings(text) for text in fact_check_corpus]

# # Check the shape of the first embedding to ensure it's consistent
# print(fact_embeddings[0].shape)  # Should print (512,)


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Compute alignment score for each English text
# english_text = data['Aug_text'].fillna("").tolist()

# fact_alignment_scores = []

# for text in english_text:
#     text_embedding = get_embeddings(text)

#     # Compute the cosine similarity for each fact-check claim
#     scores = [cosine_similarity([text_embedding], [fact])[0][0] for fact in fact_embeddings]
#     fact_alignment_scores.append(max(scores))  # Get the highest alignment score

# # Add the fact alignment scores to your dataset
# data['Fact_Alignment_Score'] = fact_alignment_scores


STEP-4: INDIC bert embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Load Indic-BERT model and tokenizer
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get embeddings
def get_embeddings(text):
    # Add max_length and truncation to tokenizer to handle long sequences
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Extract embeddings for Hindi text
hindi_embeddings = [get_embeddings(text) for text in data['translated_text'].fillna("")]
hindi_embeddings = np.vstack(hindi_embeddings)

# Add embeddings to the dataframe
for i in range(hindi_embeddings.shape[1]):
    data[f'Hindi_Embedding_{i}'] = hindi_embeddings[:, i]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentiment polarity

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the model and tokenizer
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"  # or any other sentiment model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to analyze sentiment
def get_sentiment_hindi(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    # Assuming the labels are: 0 = Negative, 1 = Neutral, 2 = Positive
    sentiment = torch.argmax(scores, dim=1).item()
    return sentiment

# Apply sentiment analysis
data['Sentiment_Label'] = data['translated_text'].fillna("").apply(get_sentiment_hindi)


STEP-5:combine all features

In [ ]:
# # Select relevant features
# # print(data.head());
# feature_columns = [col for col in data.columns if col.startswith('LDA_Topic') or
#                    col.startswith('Hindi_Embedding') or col in ['NER_Count', 'Sentiment_Label']]

# X = data[feature_columns]
# y = data['Label']


STEP-6: split data for testing and training

In [ ]:
# from sklearn.model_selection import train_test_split

# # Drop rows with NaN values in the target variable 'Label'
# data = data.dropna(subset=['Label'])

# # Split the dataset after removing NaN values
# X_train, X_test, y_train, y_test = train_test_split(data[feature_columns], data['Label'], test_size=0.2, random_state=42)


STEP-7: train the model

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# # Define and train the Random Forest classifier
# clf = RandomForestClassifier(n_estimators=1000, random_state=42)
# clf.fit(X_train, y_train)


STEP-8: evaluate the model

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# # Predict on test data
# y_pred = clf.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# print(f"Accuracy: {accuracy:.2f}")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1-Score: {f1:.2f}")

support vector machine

In [ ]:
# Required Libraries
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load your dataset (assume `data` is preprocessed)
# Replace `feature_columns` and `Label` with actual column names
feature_columns = [col for col in data.columns if col.startswith('LDA_Topic') or
                   col.startswith('Hindi_Embedding') or col in ['NER_Count', 'Sentiment_Label']]
X = data[feature_columns]
y = data['Label']
data = data.dropna(subset=['Label'])
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features for better performance of SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 1: Hyperparameter Tuning using GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.001, 0.01, 0.1, 1]
}
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train_scaled, y_train)

# Best Parameters and Best Score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Step 2: Cross-Validation Scores
cv_scores = cross_val_score(grid_search.best_estimator_, X_train_scaled, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", np.mean(cv_scores))

# Step 3: Evaluate Model on Test Data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.decision_function(X_test_scaled)  # For ROC-AUC

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test Accuracy: {accuracy:.2f}")
print(f"ROC-AUC Score: {roc_auc:.2f}")

# Step 4: ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

# Step 5: Interpretability with SHAP
explainer = shap.Explainer(best_model, X_train_scaled)
shap_values = explainer(X_test_scaled)

# Summary Plot
shap.summary_plot(shap_values, X_test, feature_names=feature_columns)

# Individual Force Plot for Example Instance
sample_idx = 0  # Change index for other samples
shap.force_plot(explainer.expected_value[1], shap_values[sample_idx].values, X_test.iloc[sample_idx],
                matplotlib=True)


Random forest

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Prepare the Data
# Assume the data contains required columns: 'LDA_Topic_*', 'Hindi_Embedding_*', 'NER_Count', 'Sentiment_Label', 'Label'

# Select feature columns (replace with your actual column names)
lda_features = data[[col for col in data.columns if col.startswith('LDA_Topic')]].values
hindi_embeddings = data[[col for col in data.columns if col.startswith('Hindi_Embedding')]].values
ner_counts = data['NER_Count'].values.reshape(-1, 1)  # Reshape for single-column feature
sentiment_labels = data['Sentiment_Label'].values.reshape(-1, 1)  # Reshape for single-column feature

# Combine features using NumPy
combined_features = np.hstack([lda_features, hindi_embeddings, sentiment_labels])

# Extract target labels
labels = data['Label'].values

# Step 2: Split the Dataset
# Drop NaN labels to avoid errors
data = data.dropna(subset=['Label'])
X_train, X_test, y_train, y_test = train_test_split(
    combined_features, labels, test_size=0.2, random_state=42, stratify=labels
)

# Step 3: Train the Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rf_model.fit(X_train, y_train)

# Step 4: Evaluate the Model
y_pred = rf_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


indic bert at final stage

In [ ]:
# from transformers import AutoTokenizer, BertForSequenceClassification
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report
# import torch
# from torch.utils.data import DataLoader, Dataset

# # Step 1: Load IndicBERT
# model_name = "ai4bharat/indic-bert"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# # Step 2: Define a Custom Dataset
# class SimpleRumorDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_length=128):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         # Tokenize text
#         tokens = self.tokenizer(
#             self.texts[idx],
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_length,
#             return_tensors="pt"
#         )
#         label = torch.tensor(self.labels[idx], dtype=torch.long)
#         return {
#             "input_ids": tokens["input_ids"].squeeze(),
#             "attention_mask": tokens["attention_mask"].squeeze(),
#             "label": label
#         }

# # Step 3: Prepare Dataset
# texts = data["translated_text"].tolist()  # Hindi text
# labels = data["Label"].values

# # Split data
# X_train_texts, X_test_texts, y_train, y_test = train_test_split(
#     texts, labels, test_size=0.2, random_state=42, stratify=labels
# )

# # Create datasets and dataloaders
# train_dataset = SimpleRumorDataset(X_train_texts, y_train, tokenizer)
# test_dataset = SimpleRumorDataset(X_test_texts, y_test, tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16)

# # Step 4: Define Training Loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# criterion = torch.nn.CrossEntropyLoss()

# def train_epoch(model, train_loader, optimizer, criterion, device):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["label"].to(device)

#         # Forward pass
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         # Backward pass
#         loss.backward()
#         optimizer.step()
#     return total_loss / len(train_loader)

# def evaluate(model, test_loader, device):
#     model.eval()
#     preds = []
#     true_labels = []
#     with torch.no_grad():
#         for batch in test_loader:
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)

#             outputs = model(input_ids, attention_mask=attention_mask)
#             logits = outputs.logits
#             predictions = torch.argmax(logits, dim=1)

#             preds.extend(predictions.cpu().numpy())
#             true_labels.extend(batch["label"].cpu().numpy())
#     return accuracy_score(true_labels, preds), classification_report(true_labels, preds)

# # Step 5: Train and Evaluate
# for epoch in range(3):  # Train for 3 epochs
#     train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
#     print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}")

# accuracy, report = evaluate(model, test_loader, device)
# print(f"Test Accuracy: {accuracy:.2f}")
# print("\nClassification Report:\n", report)
